In [180]:
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression

confirmed_cases=pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths = pd.read_csv('csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

from bokeh.plotting import show, output_notebook, figure
from bokeh.models import ColumnDataSource, LogScale, DatetimeAxis, DatetimeTicker, DaysTicker,Label
output_notebook()

cases = confirmed_cases.groupby('Country/Region').sum().iloc[:,2:].transpose()
cases['total'] = cases.sum(axis=1)

cases.index=pd.to_datetime(cases.index)
cases.index.name='Date'
from sklearn.linear_model import LinearRegression

cases = cases/100
source = ColumnDataSource(cases)
recent_US = np.log(cases['US'][datetime.date.today()-datetime.timedelta(days=8):])
L = LinearRegression().fit(np.array(range(len(recent_US))).reshape(-1,1),recent_US)
percent_rate = 100*(np.exp(L.coef_)-1)[0]

F = figure(width=600,height=600,x_axis_type='datetime',y_axis_type='log',y_range=(.1,2000),x_range=(datetime.datetime(2020,2,1),datetime.datetime(2020,4,1)))


F.line(x='Date',y='US',source=source,line_width=3,color='darkblue',legend_label='US')
F.line(x='Date',y='Italy',source=source,line_width=3,color='green',legend_label='Italy')
F.line(x='Date',y='Germany',source=source, line_width=3,color='green',line_dash='dotted',legend_label='Germany')
F.line(x='Date',y='China',source=source,line_width=3,color='orange',legend_label='China')
#F.line(x='Date',y='predicted',color='red',line_width=3,source=source,line_dash='dotted')
F.line(x='Date',y='total',color='black',line_width=3,line_dash='dashed',source=source,legend_label='World')
F.title.text = "COVID-19 Confirmed Cases (https://github.com/CSSEGISandData/COVID-19)"
F.add_layout(Label(text_font_style='bold',text_font_size="10pt",x=300,y=100,x_units='screen',y_units='screen',text='US Daily Growth Rate: {:.0f} %'.format(percent_rate)))
F.yaxis.axis_label = "100's of cases (log scale)"
F.xaxis.axis_label="Date"
F.background_fill_color="#EEEEEE"
F.xgrid.grid_line_color="white"
F.ygrid.grid_line_color="white"
show(F)